In [1]:
import numpy
import pandas
import os
import hsml
from hsml.client.exceptions import RestAPIError

Starting Spark application


ID,Application ID,Kind,State,Spark UI,Driver log
4,application_1643795132046_0005,pyspark,idle,Link,Link


SparkSession available as 'spark'.


In [2]:
def setup_env():    

    connection = hsml.connection()
    mr = connection.get_model_registry()
        
    return mr

In [3]:
mr = setup_env()

Connected. Call `.close()` to terminate connection gracefully.

In [4]:
# Test metrics, description, model_schema and input examples

In [5]:
exported_tf_model = mr.get_model("model_tf")

In [6]:
assert isinstance(exported_tf_model, hsml.tensorflow.model.Model)

assert 'accuracy' in exported_tf_model.training_metrics and 'loss' in exported_tf_model.training_metrics

assert exported_tf_model.description == "A test desc for this model"
print(exported_tf_model.model_schema)
# Check input_example and model schema
assert len(exported_tf_model.model_schema['input_schema']['columnar_schema']) == 3, "schema len incorrect"
assert exported_tf_model.model_schema['output_schema']['tensor_schema']['type'] == "float64", "schema type incorrect"
assert exported_tf_model.model_schema['output_schema']['tensor_schema']['shape'] == '(8,)', "schema shape incorrect"
assert len(exported_tf_model.input_example) == 3, "input example len incorrect"

{'input_schema': {'columnar_schema': [{'name': 'int_column', 'type': 'int64'}, {'name': 'string_column', 'type': 'object'}, {'name': 'float_column', 'type': 'float64'}]}, 'output_schema': {'tensor_schema': {'shape': '(8,)', 'type': 'float64'}}}

In [7]:
exported_tf_model_v3 = mr.get_model("model_tf", version=3)
assert exported_tf_model_v3.version == 3, "Model version should be 3"

In [8]:
try:
    skl_model = mr.get_model("not_found")
    assert False, "should return RestAPIError"
except RestAPIError:
    pass

In [9]:
skl_model = mr.get_model("model_sklearn")

In [10]:
skl_model.delete()

In [11]:
tf_models = mr.get_models("model_tf")

In [12]:
assert len(tf_models) == 3

In [13]:
best_tf_model = mr.get_best_model("model_tf", "accuracy", "max")

In [14]:
assert best_tf_model.version == 2, "Highest accuracy should be version 2"

In [ ]:
model_dir = best_tf_model.download()

In [ ]:
assert 'saved_model.pb' in os.listdir(model_dir), "Model file should be in the downloaded model directory"